# Linear Regression — Standout Implementation

**Objective:** Implement simple & multiple linear regression on a housing dataset, do advanced diagnostics, and produce a professional README-quality notebook suitable for GitHub.

**How to use:** Download the housing dataset (e.g., `housing.csv`) and place it in the same folder as this notebook. If you use Kaggle, download and rename the file to `housing.csv` or `dataset.csv`.

**Contents:**
- EDA & preprocessing
- Feature engineering & selection
- Simple and multiple linear regression (scikit-learn)
- Statsmodels OLS summary for statistical insight
- Regularization (Ridge, Lasso, ElasticNet)
- Multicollinearity check (VIF)
- Diagnostics & plots (residuals, Q-Q)
- Model interpretability (coefficients, SHAP fallback)
- Save model with `joblib`


In [ ]:
# Imports
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import statsmodels.api as sm

from joblib import dump

print('Libraries imported successfully')

In [ ]:
# Load dataset - update filename if needed
for fname in ('housing.csv', 'dataset.csv', 'data.csv'):
    if os.path.exists(fname):
        DATA_PATH = fname
        break
else:
    DATA_PATH = None

if DATA_PATH is None:
    print('No dataset found in working directory.\nPlease download the housing dataset (Kaggle link provided in the assignment) and place it as `housing.csv` here.')
else:
    print('Loading dataset from', DATA_PATH)
    df = pd.read_csv(DATA_PATH)
    print('Shape:', df.shape)
    display(df.head())

In [ ]:
# Basic EDA (run after loading the dataset)
try:
    df
except NameError:
    raise RuntimeError('Dataset not loaded. Run the dataset cell and ensure file is present.')

print('\n--- Data Info ---')
df.info()

print('\n--- Missing values (per column) ---')
print(df.isnull().sum().sort_values(ascending=False).head(20))

print('\n--- Descriptive statistics ---')
display(df.describe().T)

# Target distribution check (assumes target named price or SalePrice - adjust if different)
possible_targets = [c for c in df.columns if 'price' in c.lower() or 'saleprice' in c.lower() or c.lower()=='price']
print('\nPossible target columns detected:', possible_targets)

if possible_targets:
    target = possible_targets[0]
else:
    # fallback to last numeric column
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    target = numeric_cols[-1]
    print('No obvious "price" column found; using numeric column', target)

plt.figure(figsize=(8,5))
sns.histplot(df[target].dropna(), kde=True)
plt.title(f'Distribution of target: {target}')
plt.show()

# Correlation heatmap for numeric features (top 20)
num_df = df.select_dtypes(include=[np.number])
if num_df.shape[1] > 1:
    corr = num_df.corr()[target].abs().sort_values(ascending=False).head(20)
    print('\nTop correlated numeric features with target:')
    display(corr)
    plt.figure(figsize=(10,8))
    sns.heatmap(num_df[corr.index].corr(), annot=False, cmap='coolwarm')
    plt.title('Correlation matrix (top numeric features)')
    plt.show()
else:
    print('Not enough numeric columns for correlation heatmap.')

In [ ]:
# Feature engineering suggestions (customize for your dataset)
# Example transformations - comment/uncomment based on dataset columns

# If dataset has 'sqft' and 'price', create price_per_sqft
def safe_add_price_per_sqft(df):
    candidates = [c for c in df.columns if 'sqft' in c.lower() or 'square' in c.lower()]
    if candidates and target in df.columns:
        col = candidates[0]
        df['price_per_sqft'] = df[target] / (df[col].replace(0, np.nan))
        print('Added price_per_sqft using', col)
    else:
        print('No sqft-like column detected; skipping price_per_sqft feature')

safe_add_price_per_sqft(df)

# Example: convert date column to year or age
date_cols = [c for c in df.columns if 'yr' in c.lower() or 'year' in c.lower() or 'date' in c.lower()]
if date_cols:
    print('Date-like columns:', date_cols)
    # Uncomment and customize as needed
    # df['age'] = 2025 - df[date_cols[0]]

print('\nAfter feature suggestions, df.columns sample:')
print(df.columns[:30])

In [ ]:
# Modeling pipeline (automated preprocessing)
# This pipeline will:
# - Separate numerical and categorical features
# - Impute missing values
# - Scale numeric features
# - One-hot encode categorical features

# Select features automatically: numeric + top-k categorical by cardinality
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_features:
    numeric_features.remove(target)

categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

print('Numeric features:', len(numeric_features))
print('Categorical features:', len(categorical_features))

# Define transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Prepare X and y
X = df[numeric_features + categorical_features].copy()
y = df[target].copy()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simple Linear Regression pipeline (as baseline)
simple_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())])

print('Fitting baseline LinearRegression...')
simple_pipeline.fit(X_train, y_train)
print('Baseline fitted')

# Predictions & metrics
y_pred = simple_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Baseline results -- MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}')

In [ ]:
# Statsmodels OLS: get design matrix after preprocessing (useful for p-values)
from sklearn.preprocessing import OneHotEncoder

# Fit preprocessor on full X and transform to a dense DataFrame
preprocessor.fit(X)

# Get transformed feature names
num_cols = numeric_features
cat_cols = []
if categorical_features:
    ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
    ohe_features = ohe.get_feature_names_out(categorical_features)
    cat_cols = list(ohe_features)

feature_names = num_cols + cat_cols

X_trans = preprocessor.transform(X)
# Convert to DataFrame (handle sparse matrix if produced)
if hasattr(X_trans, 'toarray'):
    X_arr = X_trans.toarray()
else:
    X_arr = X_trans
X_design = pd.DataFrame(X_arr, columns=feature_names)
X_design = sm.add_constant(X_design)

print('Design matrix shape for statsmodels:', X_design.shape)

# Fit OLS (may be slow for large matrices)
ols_model = sm.OLS(y, X_design).fit()
print(ols_model.summary())

In [ ]:
# Variance Inflation Factor (VIF) calculation
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Compute VIF on numeric part only (after imputation/scaling)
X_num = preprocessor.named_transformers_['num'].transform(df[numeric_features])
X_num_df = pd.DataFrame(X_num, columns=numeric_features)

vif_data = pd.DataFrame()
vif_data['feature'] = X_num_df.columns
vif_data['VIF'] = [variance_inflation_factor(X_num_df.values, i) for i in range(X_num_df.shape[1])]

print('VIF for numeric features:')
display(vif_data.sort_values('VIF', ascending=False).head(20))

In [ ]:
# Regularized regression: Ridge, Lasso, ElasticNet (with cross-validation recommended)
models = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5)
}

results = {}
for name, model in models.items():
    pipe = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test, preds),
        'MSE': mean_squared_error(y_test, preds),
        'RMSE': np.sqrt(mean_squared_error(y_test, preds)),
        'R2': r2_score(y_test, preds)
    }

results_df = pd.DataFrame(results).T
print('Model comparison:')
display(results_df)

In [ ]:
# Extract coefficients from linear model after preprocessing
coef_pipeline = simple_pipeline
pre = coef_pipeline.named_steps['preprocessor']
reg = coef_pipeline.named_steps['regressor']

# Build feature names again
num_cols = numeric_features
cat_cols = []
if categorical_features:
    ohe = pre.named_transformers_['cat'].named_steps['onehot']
    cat_cols = list(ohe.get_feature_names_out(categorical_features))

all_feat_names = num_cols + cat_cols
coefs = reg.coef_

coef_df = pd.DataFrame({'feature': all_feat_names, 'coefficient': coefs})
coef_df = coef_df.reindex(coef_df.coefficient.abs().sort_values(ascending=False).index)

print('Top coefficients:')
display(coef_df.head(20))

plt.figure(figsize=(8,6))
sns.barplot(data=coef_df.head(20), x='coefficient', y='feature')
plt.title('Top 20 feature coefficients (baseline linear regression)')
plt.show()

In [ ]:
# Residual diagnostics
preds = simple_pipeline.predict(X_test)
residuals = y_test - preds

plt.figure(figsize=(8,5))
sns.scatterplot(x=preds, y=residuals)
plt.axhline(0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted')
plt.show()

# Q-Q plot for residuals
sm.qqplot(residuals, line='s')
plt.title('Q-Q plot of residuals')
plt.show()

In [ ]:
# SHAP - optional (if shap is installed). Provides local & global interpretability.
try:
    import shap
    # For pipeline, explain using transformed features may be needed
    X_train_trans = preprocessor.transform(X_train)
    explainer = shap.Explainer(simple_pipeline.named_steps['regressor'], X_train_trans)
    X_test_trans = preprocessor.transform(X_test)
    shap_values = explainer(X_test_trans)
    print('SHAP available — showing summary plot (may be slow)')
    shap.summary_plot(shap_values, features=X_test_trans, feature_names=(num_cols + cat_cols))
except Exception as e:
    print('SHAP not available or failed to run:', e)
    print('You can install shap via pip to enable this section: pip install shap')

In [ ]:
# Save the baseline model for deployment
os.makedirs('saved_model', exist_ok=True)
dump(simple_pipeline, 'saved_model/linear_model_baseline.joblib')
print('Saved baseline model to saved_model/linear_model_baseline.joblib')

## Next steps & suggestions for the README

- Hyperparameter tuning for Ridge/Lasso/ElasticNet using `GridSearchCV` or `RandomizedSearchCV`.
- Try tree-based models (RandomForest, XGBoost) as benchmarks and compare performance.
- Use MLflow or Weights & Biases to track experiments and metrics.
- Deploy a Streamlit app (`streamlit run app.py`) that loads the saved model and offers a prediction form.

---

**When submitting to GitHub:** include `README.md`, the notebook `linear_regression_notebook.ipynb`, `saved_model/linear_model_baseline.joblib` (or provide link to model), and a sample `dataset.csv` or instructions to download it from Kaggle.

Good luck! If you'd like, I can also: 
- generate the `README.md` for the repository,
- create a Streamlit app skeleton to deploy the model,
- or run this notebook here if you upload the dataset file.
